In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import hashlib
import random
import os
import pickle
import joblib
os.chdir('Resources/')

In [2]:
def load_key(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [3]:
def string_to_int(message):
    return int.from_bytes(message.encode('utf-8'), byteorder='big')

def int_to_string(value):
    byte_length = (value.bit_length() + 7) // 8 
    return value.to_bytes(byte_length, byteorder='big').decode('utf-8', errors='ignore')

def float_to_string(message):
    return str(message)

def string_to_float(message):
    return float(message)

In [4]:
def power(b, p, m):
    b = b % m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

In [5]:
def mod_inv(a, m):
    origin_m = m
    y, x = 0, 1

    if m == 1:
        return 0

    while a > 1:
        q = a // m
        temp = m

        m = a % m
        a = temp
        temp = y

        y = x - q * y
        x = temp

    if x < 0:
        x += origin_m
    return x

In [6]:
df = pd.read_csv('20_(SYS)_CC_Received_User_Data.csv')
#df = df.dropna()
df1 = df.iloc[:, :5]
df2 = df.iloc[:, 5:10]
df3 = df.iloc[:, 10:]  
df3

,ST_Slope,HeartDisease
0,Upsloping,NaN


In [7]:
def encrypt_data(n, df, public_keys, column_names):
    encrypted_data = pd.DataFrame()

    for k in range(n):
        public_key = public_keys[column_names[k]] 
        p = public_key['p']
        g = public_key['g']
        h = public_key['h']
        y = public_key['y']
            
        print(f"Processing Column: {column_names[k]}")

        s = power(h, y, p)

        messages = df.iloc[:, k].values 

        encrypted_column = []

        for i, message in enumerate(messages, 1):
            if message is not None and message == message:  
                print(f"Processing Message {i}: {message}")
                
                temp_message = string_to_int(message)  
                
                c1 = power(g, y, p)
                c2 = (temp_message * s) % p
                
                encrypted_column.append(f"({c1},{c2})") 
                print(f"Ciphertext: (c1, c2): ({c1}, {c2})")
            
            else:
                encrypted_column.append(None)

        encrypted_data[column_names[k]] = encrypted_column

    return encrypted_data

In [8]:
public_keys = load_key("5_1_CR1_Public_Key_Part.pkl")

column_names = [
    "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol"
]

encrypted_data_1 = encrypt_data(5, df1, public_keys, column_names)
encrypted_data_1

Processing Column: Age
Processing Message 1: Middle-aged
Ciphertext: (c1, c2): (7725238297445332127407991595158267606450716631553, 4274730442883565920336037254176546269980471338891)
Processing Column: Sex
Processing Message 1: Female
Ciphertext: (c1, c2): (35517718033104704088502485275726341561551904284337, 23188915792823836432044653809327381448450200244661)
Processing Column: ChestPainType
Processing Message 1: Atypical Angina
Ciphertext: (c1, c2): (22443659489472860229040290952583046495774185357615, 7241693035070086510586471683043736059438700931885)
Processing Column: RestingBloodPressure
Processing Message 1: Normal
Ciphertext: (c1, c2): (32645786427389652046014896610704699713743820311933, 12861701305792585207367963936785195559926271406427)
Processing Column: Cholesterol
Processing Message 1: High
Ciphertext: (c1, c2): (1304267054805981944467088018208132613735969340595, 42465400876689406566951597473927995615875243628626)


,Age,Sex,ChestPainType,RestingBloodPressure,Cholesterol
0,(772523829744533212740799159515826760645071663...,(355177180331047040885024852757263415615519042...,(224436594894728602290402909525830464957741853...,(326457864273896520460148966107046997137438203...,(130426705480598194446708801820813261373596934...


In [9]:
public_keys = load_key("5_2_CR2_Public_Key_Part.pkl")

column_names = [
    "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina", "Oldpeak"
]

encrypted_data_2 = encrypt_data(5, df2, public_keys, column_names)
encrypted_data_2

Processing Column: FastingBloodSugar
Processing Message 1: No
Ciphertext: (c1, c2): (18959108316266119489233644530980633950698663352542, 33810995375882583283482249513236475170709139984741)
Processing Column: RestingECG
Processing Message 1: Normal
Ciphertext: (c1, c2): (12741437569542050747073334897738270095511225090014, 4937776636611353583684620223178330241268700879095)
Processing Column: MaximumHeartRate
Processing Message 1: Moderate
Ciphertext: (c1, c2): (7384466023307569122962762536709314464704923088871, 12957944149271503600812266143018628298526313693571)
Processing Column: ExerciseAngina
Processing Message 1: No
Ciphertext: (c1, c2): (20461355034696913938060735031266347179123151849021, 34978493924221736660734634798859756197578254899277)
Processing Column: Oldpeak


,FastingBloodSugar,RestingECG,MaximumHeartRate,ExerciseAngina,Oldpeak
0,(189591083162661194892336445309806339506986633...,(127414375695420507470733348977382700955112250...,(738446602330756912296276253670931446470492308...,(204613550346969139380607350312663471791231518...,None


In [10]:
public_keys = load_key("5_3_CC_Public_Key_Part.pkl")

column_names = [
    "ST_Slope"
]

encrypted_data_3 = encrypt_data(1, df3, public_keys, column_names)
encrypted_data_3

Processing Column: ST_Slope
Processing Message 1: Upsloping
Ciphertext: (c1, c2): (33269922789787438881901292310454632547714019546100, 5701636905427024824662866983306169439856314483175)


,ST_Slope
0,(332699227897874388819012923104546325477140195...


In [11]:
encrypted_df = pd.concat([encrypted_data_1, encrypted_data_2, encrypted_data_3], axis='columns')
encrypted_df
encrypted_df.to_csv('21_(SYS)_CR_Encrypted_User_Data.csv', index=False)